# Алгоритм и программа построения интерполяционного полинома Ньютона

# Задание
1. Задана таблица значений функции вида x, y, 𝑓(𝑥, y). С помощью интерполяции, используя
метод полинома Ньютона, найти приближённое значение функции Z(X, Y) от введённого 𝑋, Y.
Ввод: значение 𝑋 и степень полинома
Вывод: значение функции от 𝑋
2. Методом обратной интерполяции найти корень функции 𝑓(𝑥) = 0. Также найти корень
методом половинного деления.

# Описание алгоритма
Для двух точек: $$y(x_i, x_j) ={ y_i - y_j \over x_i - x_j}$$

Для трёх точек: $$y(x_i, x_j, x_k) ={ y(x_i, x_j) - y(x_j, x_k) \over x_i - x_k}$$

Для n точек: $$y(x_i, x_j, ..., x_n) ={ y(x_i, x_j, ..., x_{n - 1}) - y(x_j, ..., x_n) \over x_i - x_n}$$

Отсюда искомый полином будет равен:
$$P_n(x) = y_0 + (x-x_0)y(x_0, x_1) + (x - x_0)(x - x_1)y(x_0, x_1, x_2) + ... \\ + (x - x_0)(x - x_1)...(x - x_{n - 1})y(x_0, x_1, ..., x_n)$$
$$P_n(x) = y_0 + \sum\limits_{k = 0}^n (x - x_0) ...(x - x_{n - 1})y(x_0,x_1,...,x_k)$$

In [2]:
import numpy as np
import math
from heapq import nsmallest

In [3]:
LEN_ERROR = 2
STEP_ERROR = -10000000
BISECTION_ERROR = -100000
EPS = 0.0001
MAX_STEP = 10000
NEWTON_ERROR = -100000

In [4]:
def make_coordinates_table(filename):
    with open(filename) as f:
        coordinates = [list(map(float, row.split())) for row in f.readlines()]
    return coordinates

In [5]:
def make_back_coordinates_table(coordinates):
    back_coordinates = list()
    for i in range(len(coordinates)):
        back_coordinates.append([coordinates[i][1], coordinates[i][0]])
    return back_coordinates

In [6]:
def make_table(coordinates, n, x, is_back):
    if (n >= len(coordinates)):
        print("Can not do polynomial")
        return LEN_ERROR
    
    data = np.array(nsmallest(n + 1, coordinates, key=lambda row: abs(row[is_back] - x)))
    data = data[data[:,is_back].argsort()]
    matrix = np.zeros((n  + 1, n))

    return np.append(data, matrix, axis=1)   

In [7]:
def fill_table(table, n):
    step = 1 
    temp_column = 2
    for count in range(n + 1):
        temp_row = 0 
        while (temp_row + step) != len(table):
            table[temp_row][temp_column] = ((table[temp_row][temp_column - 1]
                                           - table[temp_row + 1][temp_column - 1]) * 1.0 /
                                            (table[temp_row][0] - table[temp_row + step][0]))
            temp_row += 1
        step += 1
        temp_column += 1
    return table
    

In [8]:
def get_polynomial(x, table, n):
    result = table[0][1]
    temp_column = 2
    for i in range (n):
        term = 1
        for j in range (temp_column - 1):
            term *= (x - table[j][0])
        term *= table[0][temp_column]
        result += term 
        temp_column += 1
    return result    

In [9]:
def newton(x, n, coordinates, is_back):
    table = make_table(coordinates, n, x, is_back)
    if type(table) != type(LEN_ERROR):
        fill_table(table, n)
        result = get_polynomial(x, table, n)
        return result
    return NEWTON_ERROR
    

In [10]:
def find_root_bisection(xa, f_xa, xb, f_xb, n, coordinates):
    step = 0
    xm = (xa + xb) / 2.0
    while (math.fabs(xb - xa) > EPS * xm + EPS * 0.1) and step < MAX_STEP:
        xm = (xa + xb) / 2.0
        f_xm = newton(xm, n, coordinates, 0)
        if f_xm != NEWTON_ERROR:
            if f_xa * f_xm <= 0:
                xb = xm
                f_xb = f_xm
            else:
                xa = xm
                f_xa = f_xm
            step += 1
        else:
            return NEWTON_ERROR
    if step >= MAX_STEP:
        return STEP_ERROR
    return (xa + xb) / 2

In [11]:
def main(x, n):
    coordinates = make_coordinates_table("coordinates.txt")
    back_coordinates = make_back_coordinates_table(coordinates)
    
    res_newton = newton(x, n, coordinates, 0)
    if (res_newton != NEWTON_ERROR):
        print("Pn(x) =", res_newton)

    res_bisection = find_root_bisection(coordinates[0][0], coordinates[0][1], coordinates[len(coordinates) - 1][0],
                                  coordinates[len(coordinates) - 1][1], n, coordinates)
    if math.fabs(res_bisection - 0) < 0.0001:
        res_bisection = 0.0
    if res_bisection != NEWTON_ERROR and res_bisection != STEP_ERROR:
        print("Root of function by bisection =", round(res_bisection, 4))
    back_res_newton = newton(0, n, back_coordinates, 1)
    if (back_res_newton != NEWTON_ERROR):
        print("Root of function by Newton =", round(back_res_newton, 4))

# Введите х и степень полинома n

In [12]:
coordinates = make_coordinates_table("coordinates.txt")
for i in coordinates:
    print (i[0], i[1])

0.0 0.0
1.0 1.0
2.0 4.0
3.0 9.0
4.0 16.0
5.0 25.0
6.0 36.0
7.0 49.0
8.0 64.0
9.0 81.0
10.0 100.0


In [13]:
x = 11
n = 1
main(x, n)

Pn(x) = 119.0
Root of function by bisection = 0.0
Root of function by Newton = 0.0
